In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'train.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg 

experiment:
  seed: '?' 
  val_first: False 
  torch_compile: True 
  torch_mixed_precision: False
    
dataloader:
  batch_size: '?' 
  num_workers: '?'
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: '?'
  
train:
  epochs: '?' 
  eval_freq: '?'

# Used for additional data experiment.
data:
  train_kwargs:
    split: 'train'
  val_kwargs: 
    split: 'val'
  
loss_func: 
  from_logits: True
  batch_reduction: 'mean' 
  # One of these most also be defined, but for reasons we won't actually have them be ?.
  # _class: ?
  # _combo_class: ?

<IPython.core.display.Javascript object>

In [3]:
%%yaml log_cfg

log:
  root: '?'
  checkpoint_freq: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg  

model:
  _class: ese.models.unet.UNet
  filters: '?'
  convs_per_block: '?' # Good default for UNets.

<IPython.core.display.Javascript object>

In [5]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [6]:
%%yaml aug_cfg

augmentations:
    spatial:
        max_translation: 5.0
        max_rotation: 5.0
        max_scaling: 1.1
        warp_integrations: 5
        warp_smoothing_range: [10, 20]
        warp_magnitude_range: [1, 3]
        affine_probability: 0.5
        warp_probability: 0.5
    visual:
        use_mask: False 
        added_noise_max_sigma: 0.01
        gamma_scaling_max: 0.1
        bias_field_probability: 0.5
        gamma_scaling_probability: 0.5
        added_noise_probability: 0.5

<IPython.core.display.Javascript object>

In [7]:
%%yaml experiment_cfg 

group: "ISLES_DebugSliter"

experiment:
    seed: 40
    seed_range: 1
    torch_compile: True

data:
    _class: "ese.datasets.ISLES"

dataloader:
    batch_size: 1
    num_workers: 1

loss_func:
    _combo_class: 
        dice_loss:
            _class: ese.losses.SoftDiceLoss
            from_logits: True
            batch_reduction: 'mean' 
            ignore_empty_labels: False 
        proportion_estimation_error:
            _class: ese.losses.AreaEstimationError
            from_logits: True
            proportion: True
            abs_diff: True
            square_diff: False
    convex_param:
        - 0.0
        - 0.1
        - 0.2
        - 0.4
        - 0.5
        - 0.6
        - 0.7
        - 0.8
        - 0.9
        - 1.0

<IPython.core.display.Javascript object>

In [8]:
from ese.analysis.analysis_utils.submit_utils import get_ese_training_configs 

# Get the configs for the different runs.
# base_cfg = Config(default_cfg).update([model_cfg, log_cfg, callbacks_cfg])
base_cfg = Config(default_cfg).update([model_cfg, log_cfg, callbacks_cfg, aug_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, train_cfgs = get_ese_training_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
len(train_cfgs)

10

## Running Jobs

In [10]:
# ####### FOR DEBUGGIN
# from ese.experiment import run_ese_exp, CalibrationExperiment

# run_ese_exp(
#     config=train_cfgs[0], 
#     experiment_class=CalibrationExperiment,
#     run_name='debug',
#     show_examples=True,
#     track_wandb=False,
#     gpu='0',
# )

In [11]:
# FOR SUBMISSION
from ese.experiment import submit_ese_exps, CalibrationExperiment 

submit_ese_exps(
    group="training",
    base_cfg=updated_base_cfg,
    exp_cfg=experiment_cfg,
    config_list=train_cfgs,
    experiment_class=CalibrationExperiment,
    track_wandb=True,
    # available_gpus=['0', '1', '2', '3'l
    available_gpus=['0', '1', '2', '3', '4', '5', '6', '7'],
)

--> Queued job-id: 10.
--> Queued job-id: 11.
--> Queued job-id: 12.
--> Queued job-id: 13.
--> Queued job-id: 14.
--> Queued job-id: 15.
--> Queued job-id: 16.
--> Queued job-id: 17.
--> Queued job-id: 18.
--> Queued job-id: 19.
